In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
warnings.filterwarnings('ignore')

In [ ]:
data=pd.read_excel("Losses.xlsx")

In [ ]:
data

### Q1. What are the total claims by year and what trends are there? What could be driving these trends?
a.What are the total value of claims by year?

b.What factors, internal and external to the insurer, could be driving any trends that you have identified?

In [ ]:
df=data.groupby(["YEAR"])["GROSS INCURRED AMOUNT"].sum()
df

### graphical representation

In [ ]:
data.groupby(["YEAR"])["GROSS INCURRED AMOUNT"].sum().plot(kind="bar",color="y")

plt.ylabel("Claim Value")
plt.title("Claims distribution per year")
plt.show()

# PIE CHART

In [ ]:
plt.figure(figsize=(9,9))
plt.title("Annual Claims Distribution")
plt.pie(x=data.groupby(["YEAR"])["GROSS INCURRED AMOUNT"].sum(),explode=[0,0,0,0, 0, 0, 0,0, 0.1],labels=[1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007],autopct='%.1f%%')
plt.show()

## b.What factors, internal and external to the insurer, could be driving any trends that you have identified?

In [ ]:
plt.figure(figsize=(9,9))
sns.barplot(x=data["YEAR"],y=data["GROSS INCURRED AMOUNT"],hue=data["CAUSE"])
plt.title("Trends Representation")
plt.show()


# Q2. Which perils should the company worry about most and why?
Which claim types (perils) have the greatest average annual frequency and severity?  


What is the average individual claim size by peril across the 9-year time period?


# Which claim types (perils) have the greatest average annual frequency and severity?

In [ ]:
tr=pd.pivot_table(data,index="CAUSE",values="DAY",aggfunc=np.mean)
tr.sort_values(by="DAY",ascending=False)

In [ ]:
tr.plot(kind="bar",color="m")
plt.title("Avg freq of claims")
plt.ylabel("No. of Occurence")
plt.show()

In [ ]:
r=pd.pivot_table(data,index="YEAR",columns="CAUSE",values="GROSS INCURRED AMOUNT",aggfunc=np.mean)
r

In [ ]:
plt.rcParams['figure.figsize'] = [10,8]
r.plot(kind="line")
plt.title("Severity of claims")
plt.ylabel("Avg claim amount")
plt.show()

In [ ]:
for i in data["CAUSE"].unique():
    print(i)
    print(data["CAUSE"].isin([i]).mean())

In [ ]:
data["CAUSE"].value_counts()

## What is the average individual claim size by peril across the 9-year time period?

In [ ]:
t=pd.pivot_table(data,index='CAUSE',values="GROSS INCURRED AMOUNT",aggfunc=np.mean)
t.sort_values(by="GROSS INCURRED AMOUNT",ascending=False)

In [ ]:
plt.figure(figsize=(13,13))
t.plot(kind="bar",color="y")
plt.title("Average Annual Frequency Distribution of claim")
plt.xlabel("CAUSE")
plt.ylabel("Avg claims")
plt.savefig("Pivot.jpg")

In [ ]:
data[(data["CAUSE"]=="WINDSTORM")&(data["MONTH"]=="January")&(data["YEAR"]==2005)&(data["DAY"]>=7)&(data["DAY"]<=9)]

### Q3. What did Windstorm Erwin cost the company? What perils drove the loss?
What was the total loss, for windstorm and other related weather perils, for Windstorm Erwin (date Jan 7th to Jan 9th, 2005)


In [ ]:
cause=data["CAUSE"].unique()
p=[]
for i in cause:
    if i!="WINDSTORM":
        p.append(data[(data["CAUSE"]==i)]["GROSS INCURRED AMOUNT"].sum())
    else:
         p.append(data[(data["CAUSE"]==i)&(data["MONTH"]=="January")&(data["YEAR"]==2005)&(data["DAY"]>=7)&(data["DAY"]<=9)]["GROSS INCURRED AMOUNT"].sum())
g={'cause':cause,'total loss':p}
data1=pd.DataFrame(g)
data1
data2=data1.iloc[[0,4,7],:]
data3=data2.set_index("cause")
data3.sort_values(by="total loss",ascending=False)


In [ ]:
data3.plot(kind="bar",color="y")
plt.title("Total loss per claim")
plt.show()

In [ ]:
data["CAUSE"].unique()

In [ ]:
x=data[["DAY","MONTH_ID","YEAR","GROSS INCURRED AMOUNT"]]

In [ ]:
x

In [ ]:
y=data[["CAUSE"]]

In [ ]:
y

In [ ]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,random_state=319)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier,GradientBoostingClassifier,RandomForestClassifier
from xgboost import XGBClassifier
gb=GradientBoostingClassifier()
dt=DecisionTreeClassifier()
rf=RandomForestClassifier()
xg=XGBClassifier()

In [ ]:
from sklearn.metrics import classification_report
def my_model(model):
    model.fit(xtrain,ytrain)
    y_pred_train=model.predict(xtrain)
    y_pred_test=model.predict(xtest)
    print('Train Data')
    print(classification_report(ytrain,y_pred_train))
    print('-'*30)
    print('Test Data')
    print(classification_report(ytest,y_pred_test))

    return model

In [ ]:
my_model(rf)

In [ ]:
my_model(dt)

In [ ]:
my_model(gb)

In [ ]:
def predict():
     day=int(input('Enter the day: '))
     month_id=float(input('Enter The Month: '))
     year=int(input('Enter the Year: ' ))
     amount=int(input('Enter the amount of money claimed: '))
     newobs=[[day,month_id,year,amount]]
     ypred=gb.predict(newobs)[0]
     return ypred

In [ ]:
predict()